# Visualizing CASP Data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [179]:
def make_training_example(lines):
    """
    Takes lines read from file and created a training example.
    """
    example = {}
    
    example['id'] = lines[1].rstrip()
    example['primary'] = lines[3].rstrip()
    
    pssm = []
    for i in range(5, 26):
        pssm.append([float(x) for x in lines[i].split()])
    example['pssm'] = pssm
    
    tertiary = []
    for i in range(27, 30):
        tertiary.append([float(x) for x in lines[i].split()])
    example['tertiary'] = tertiary
    
    example['mask'] = lines[31].rstrip()
    
    return example

In [180]:
def get_tertiary_matrix(tertiary, i):
    x = tertiary[0][3 * i], tertiary[0][3 * i + 1], tertiary[0][3 * i + 2]
    y = tertiary[1][3 * i], tertiary[1][3 * i + 1], tertiary[1][3 * i + 2]
    z = tertiary[2][3 * i], tertiary[2][3 * i + 1], tertiary[2][3 * i + 2]
    return x, y, z

In [182]:
with open('data/casp11/training_100', 'r') as raw_data:
    lines = []
    flag = False
    for line in raw_data:
        lines.append(line)
        if len(lines) == 33:
            example = make_training_example(lines)
            lines = []
    
        dssp_file = example['id'][:4].lower() + '.dssp'
        with open('data/dssp/' + dssp_file, 'r') as raw_dssp:
            secondary = [''] * len(example['primary'])
            flag = False
            for line in raw_dssp:
                if flag or ('#' in line):
                    if not flag:
                        flag = True
                        continue
                if flag:
                    if line[7:10] == '   ':
                        continue
                    i, d = int(line[7:10]), line[16]
                    if i >= len(example['primary']):
                        break
                    if d == ' ':
                        d = 'C'
                    secondary[i-1] = d
            else:
                example['secondary'] = ''.join(secondary)
                with open('data/processed/test.txt', 'a') as out_file:
                    out_file.write('id')
                    out_file.write(example['id'])
                    out_file.write('primary')
                    out_file.write(example['primary'])
                    out_file.write('pssm')
                    out_file.write(example['pssm']) # Needs to go from list of lists of floats to multiline string
                    out_file.write('secondary')
                    out_file.write(example['secondary'])
                    out_file.write('tertiary')
                    out_file.write(example['tertiary']) # Needs to go from list of lists of floats to multiline string
                    out_file.write('mask')
                    out_file.write(example['mask'])
        break

In [184]:
print(len(example))

6


In [187]:
print(len(example['primary']), len(example['secondary']))

200 146


In [188]:
print(example['mask'], example['mask'].count('-'), example['mask'].count('+'))

-------------------------++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++------------------------------- 56 144


In [200]:
print(sum(example['pssm'][:, 23]) - example['pssm'][:, 23][-1])

1.0000000000000004
